#Santa 2023 - The Polytope Permutation Puzzle

A permutation puzzle comprises a solution state, an initial state, and a set of allowed moves.

The solution state and initial state are arrangements of symbols we call colors, while the moves for a puzzle correspond to certain permutations of these arrangements.

A sequence of moves solves a puzzle if applying each permutation in the sequence to the puzzle's initial_state results in the puzzle's solution_state, and we call such a sequence a solution to the puzzle.

Additionally, a few of the elves have discovered that they can rearrange some of the stickers on a puzzle instead of applying moves, and so a puzzle may also be allotted wildcards. In this case, the resulting state may differ up to the puzzle's num_wildcards and the sequence will still be considered a solution.

The overall score for a submission is the total number of moves in all of its puzzle solutions. The goal of the competition is to solve all of the given puzzles in the fewest moves.

You may view the evaluation metric here: Santa 2023 Metric(https://www.kaggle.com/code/metric/santa-2023-metric/).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Santa_2023/

/content/drive/My Drive/Santa_2023


# Dataset Load & pre-processing

#### puzzle_info.csv
- puzzle_type - Identifies the type of puzzle. Puzzles of the same type have a common set of moves.
- allowed_moves - Describes the moves allowed in solutions of this puzzle type. Each move represents a permutation given in array form. You are also allowed to use the inverses of these moves in your puzzle solutions.


#### puzzle.csv
- id - A unique identifier for each puzzle.
- puzzle_type - Corresponding to puzzle_info.csv.
- solution_state - An arrangement of "colors" describing the solved state of the puzzle, with a semicolon ; delimiter.
- initial_state - An arrangement of colors describing the initial state of the puzzle, with a semicolon ; delimiter. A solution to a puzzle must transform the initial state to the solved state through a sequential application of the puzzle's allowed_moves.
- num_wildcards - The number of "mistakes" allowed in the final state of a solution.

#### sample_submission.csv
- A submission file in the correct format.
- id - Corresponding to puzzles.csv.
- moves - An initial, unoptimized solution.

In [ ]:
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation

In [ ]:
puzzle_info = pd.read_csv('puzzle_info.csv')
puzzle_info

,puzzle_type,allowed_moves
0,cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
1,cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ..."
2,cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
3,cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
4,cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
5,cube_7/7/7,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
6,cube_8/8/8,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
7,cube_9/9/9,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
8,cube_10/10/10,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
9,cube_19/19/19,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."


In [ ]:
puzzle_info['puzzle_type_category'] = puzzle_info['puzzle_type'].apply(lambda x: ''.join(filter(str.islower, x)))
puzzle_info['puzzle_type_category'].value_counts()

cube      11
globe      9
wreath     6
Name: puzzle_type_category, dtype: int64

In [ ]:
puzzle_info = pd.read_csv('puzzle_info.csv', index_col='puzzle_type')
puzzle_info

,allowed_moves
puzzle_type,
cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ..."
cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_7/7/7,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_8/8/8,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_9/9/9,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_10/10/10,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."


In [ ]:
# 'allowed_moves' 열의 각 행에 있는 문자열을 파이썬 객체로 변환하여 열을 갱신
puzzle_info['allowed_moves'] = puzzle_info['allowed_moves'].apply(literal_eval)
puzzle_info

,allowed_moves
puzzle_type,
cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ..."
cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_7/7/7,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_8/8/8,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_9/9/9,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
cube_10/10/10,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."


In [ ]:
puzzles = pd.read_csv('puzzles.csv', index_col='id')
puzzles

,puzzle_type,solution_state,initial_state,num_wildcards
id,,,,
0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0
...,...,...,...,...
393,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0
394,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0
395,globe_3/33,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0


In [ ]:
# ;을 기준으로 분할하여 새롭게 assign
puzzles = puzzles.assign(initial_state=lambda df: df['initial_state'].str.split(';'),
                         solution_state=lambda df: df['solution_state'].str.split(';')
                         )

In [ ]:
puzzles

,puzzle_type,solution_state,initial_state,num_wildcards
id,,,,
0,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[D, E, D, A, E, B, A, B, C, A, C, A, D, C, D, ...",0
1,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[D, E, C, B, B, E, F, A, F, D, B, F, F, E, B, ...",0
2,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[E, F, C, C, F, A, D, D, B, B, A, F, E, B, C, ...",0
3,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[A, C, E, C, F, D, E, D, A, A, F, A, B, D, B, ...",0
4,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[E, D, E, D, A, E, F, B, A, C, F, D, F, D, C, ...",0
...,...,...,...,...
393,globe_3/33,"[A, A, A, A, A, A, C, C, C, C, C, C, E, E, E, ...","[D, D, L, A, P, E, R, U, U, C, S, R, J, B, E, ...",0
394,globe_3/33,"[A, A, A, A, A, A, C, C, C, C, C, C, E, E, E, ...","[V, L, N, G, B, V, R, E, H, A, K, S, I, N, G, ...",0
395,globe_3/33,"[N0, N1, N2, N3, N4, N5, N6, N7, N8, N9, N10, ...","[N12, N219, N227, N198, N4, N208, N214, N245, ...",0


## Permutation Puzzles
The puzzle starts out with an initial state of colors and must be permuted through a sequence of moves to its solution state.

The goal is to do this with as few moves as possible.

In [ ]:
# Here is a very simple puzzle
solution_state = ['R', 'G', 'B']
initial_state = ['B', 'G', 'R']
moves = {'r': [1, 2, 0],
         's': [1, 0, 2]
         }

r = moves['r']
s = moves['s']

In [ ]:
initial_state_np = np.asarray(initial_state)
initial_state_np

array(['B', 'G', 'R'], dtype='<U1')

In [ ]:
# 1번째에 접근해서 'G', 2번째에 접근해서 'R', 0번째에 접근해서 'B'순서로 배열된다.
initial_state_np[r]

array(['G', 'R', 'B'], dtype='<U1')

In [ ]:
# You are also allowed to use the inverse of any of a puzzle's moves.
# The inverse of a permutation just applies the change with "the arrows reversed".
rp = Permutation(r)
sp = Permutation(s)

# Use np.argsort to get the inverse using array form
r_inv = np.argsort(r).tolist()
s_inv = np.argsort(s).tolist()
print(f"{r_inv=}, {s_inv=}\n")

# Use a negative power to get the inverse of a Permutation
rp_inv = rp ** -1
sp_inv = sp ** -1

# It's the same permutation either way
assert Permutation(r_inv) == rp_inv
assert Permutation(s_inv) == sp_inv

# In this case, s is equal to its inverse
assert (s == s_inv) and (sp == sp_inv)
# But r is not
assert (r != r_inv) and (rp != rp_inv)

# Inversion reverses the arrows
print('r:', rp_inv, "\tSends 1 -> 0, 2 -> 1, and 0 -> 2.")
print('s:', sp, "\tSends 1 -> 0, 0 -> 1, and 2 stays fixed." )

r_inv=[2, 0, 1], s_inv=[1, 0, 2]

r: (0 2 1) 	Sends 1 -> 0, 2 -> 1, and 0 -> 2.
s: (2)(0 1) 	Sends 1 -> 0, 0 -> 1, and 2 stays fixed.


In [ ]:
# Using array form
state = np.asarray(initial_state)
state = state[s]
state = state[r_inv]
state = state.tolist()
assert state == solution_state

# Using Permutations
state = sp(initial_state)
state = rp_inv(state)
assert state == solution_state

## Cube Puzzles
There are three puzzle types: cube, wreath, and globe.

Each type of puzzle represents its arrangements on some geometric figure with the permutations being a twist or turn of some portion of the figure.

In [ ]:
# Convert a state list to a dictionary of labeled faces
def cube_state_to_faces(state):
    n = int(np.sqrt(len(state) / 6))  # cube_n/n/n
    n2 = n ** 2

    labels = f"d{n-1},f0,r0,f{n-1},r{n-1},d0".split(',')
    faces = {}
    for i, l in enumerate(labels):
        face = state[n2 * i : n2 * (i + 1)]
        faces[l] = np.asarray(face).reshape(n, n).tolist()

    return faces

In [ ]:
for ptype in ('cube_2/2/2', 'cube_3/3/3'):
    sstate = puzzles.query(f"puzzle_type == '{ptype}'").iloc[0, 1]
    print(ptype)

    pprint(cube_state_to_faces(sstate))
    print()

cube_2/2/2
{'d0': [['F', 'F'], ['F', 'F']],
 'd1': [['A', 'A'], ['A', 'A']],
 'f0': [['B', 'B'], ['B', 'B']],
 'f1': [['D', 'D'], ['D', 'D']],
 'r0': [['C', 'C'], ['C', 'C']],
 'r1': [['E', 'E'], ['E', 'E']]}

cube_3/3/3
{'d0': [['F', 'F', 'F'], ['F', 'F', 'F'], ['F', 'F', 'F']],
 'd2': [['A', 'A', 'A'], ['A', 'A', 'A'], ['A', 'A', 'A']],
 'f0': [['B', 'B', 'B'], ['B', 'B', 'B'], ['B', 'B', 'B']],
 'f2': [['D', 'D', 'D'], ['D', 'D', 'D'], ['D', 'D', 'D']],
 'r0': [['C', 'C', 'C'], ['C', 'C', 'C'], ['C', 'C', 'C']],
 'r2': [['E', 'E', 'E'], ['E', 'E', 'E'], ['E', 'E', 'E']]}



In [ ]:
print("cube_2/2/2")
for m, p in puzzle_info.loc['cube_2/2/2', 'allowed_moves'].items():
    print(f"{m}: {Permutation(p)}")

print()

print("cube_3/3/3")
for m, p in puzzle_info.loc['cube_3/3/3', 'allowed_moves'].items():
    print(f"{m}: {Permutation(p)}")

cube_2/2/2
f0: (23)(2 19 21 8)(3 17 20 10)(4 6 7 5)
f1: (0 18 23 9)(1 16 22 11)(12 13 15 14)
r0: (1 5 21 14)(3 7 23 12)(8 10 11 9)
r1: (23)(0 4 20 15)(2 6 22 13)(16 17 19 18)
d0: (6 18 14 10)(7 19 15 11)(20 22 23 21)
d1: (23)(0 1 3 2)(4 16 12 8)(5 17 13 9)

cube_3/3/3
f0: (53)(6 44 47 18)(7 41 46 21)(8 38 45 24)(9 15 17 11)(10 12 16 14)
f1: (53)(3 43 50 19)(4 40 49 22)(5 37 48 25)
f2: (0 42 53 20)(1 39 52 23)(2 36 51 26)(27 29 35 33)(28 32 34 30)
r0: (2 11 47 33)(5 14 50 30)(8 17 53 27)(18 24 26 20)(19 21 25 23)
r1: (53)(1 10 46 34)(4 13 49 31)(7 16 52 28)
r2: (53)(0 9 45 35)(3 12 48 32)(6 15 51 29)(36 38 44 42)(37 41 43 39)
d0: (15 42 33 24)(16 43 34 25)(17 44 35 26)(45 51 53 47)(46 48 52 50)
d1: (53)(12 39 30 21)(13 40 31 22)(14 41 32 23)
d2: (53)(0 2 8 6)(1 5 7 3)(9 36 27 18)(10 37 28 19)(11 38 29 20)


## Wreath Puzzles
A wreath puzzle is two rings joined at two points

In [ ]:
# A wreath with six in the left and six in the right
# You can see the rings join at points 0 and 2.
print("wreath_6/6")
for m, p in puzzle_info.loc['wreath_6/6', 'allowed_moves'].items():
    print(f"{m}: {Permutation(p)}")

wreath_6/6
l: (9)(0 1 2 3 4 5)
r: (0 6 7 2 8 9)


## Globe Puzzles
A globe puzzle is a sphere with cuts along lines of latitude and longitude.

If you poke a hole at the North and South poles, cut along the meridian, and spread it out, it will look like a grid:

In [ ]:
# Get a solution_state
ss_globe34 = puzzles.query("puzzle_type == 'globe_3/4'").iloc[0, 1]
ss_globe34

['A',
 'A',
 'C',
 'C',
 'E',
 'E',
 'G',
 'G',
 'A',
 'A',
 'C',
 'C',
 'E',
 'E',
 'G',
 'G',
 'B',
 'B',
 'D',
 'D',
 'F',
 'F',
 'H',
 'H',
 'B',
 'B',
 'D',
 'D',
 'F',
 'F',
 'H',
 'H']

In [ ]:
# Reshape into a grid
ss_globe34 = np.asarray(ss_globe34).reshape(3+1, 2*4)
ss_globe34

array([['A', 'A', 'C', 'C', 'E', 'E', 'G', 'G'],
       ['A', 'A', 'C', 'C', 'E', 'E', 'G', 'G'],
       ['B', 'B', 'D', 'D', 'F', 'F', 'H', 'H'],
       ['B', 'B', 'D', 'D', 'F', 'F', 'H', 'H']], dtype='<U1')

In [ ]:
print("globe_3/4")
for m, p in puzzle_info.loc['globe_3/4', 'allowed_moves'].items():
    print(f"{m}: {Permutation(p)}")

globe_3/4
r0: (31)(0 1 2 3 4 5 6 7)
r1: (31)(8 9 10 11 12 13 14 15)
r2: (31)(16 17 18 19 20 21 22 23)
r3: (24 25 26 27 28 29 30 31)
f0: (31)(0 27)(1 26)(2 25)(3 24)(8 19)(9 18)(10 17)(11 16)
f1: (31)(1 28)(2 27)(3 26)(4 25)(9 20)(10 19)(11 18)(12 17)
f2: (31)(2 29)(3 28)(4 27)(5 26)(10 21)(11 20)(12 19)(13 18)
f3: (31)(3 30)(4 29)(5 28)(6 27)(11 22)(12 21)(13 20)(14 19)
f4: (4 31)(5 30)(6 29)(7 28)(12 23)(13 22)(14 21)(15 20)
f5: (0 29)(5 24)(6 31)(7 30)(8 21)(13 16)(14 23)(15 22)
f6: (0 31)(1 30)(6 25)(7 24)(8 23)(9 22)(14 17)(15 16)
f7: (0 25)(1 24)(2 31)(7 26)(8 17)(9 16)(10 23)(15 18)


## Submissions

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv', index_col='id')
sample_submission

,moves
id,
0,r1.-f1
1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...
393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....


In [ ]:
# Apply a sequence of moves in array form to a color state.
def apply_sequence(sequence, moves, state):
    state = np.asarray(state)
    for m in sequence.split('.'):
        state = state[moves[m]]

    return state

In [ ]:
# Convert allowed_moves to dict and add inverse moves
all_moves = puzzle_info.loc[:, 'allowed_moves'].to_dict()
for ptype, moves in all_moves.copy().items():
    for m, arr in moves.copy().items():
        all_moves[ptype][f"-{m}"] = np.argsort(arr).tolist()

# Get info for the first puzzle
solution_state = puzzles.iloc[0, 1]
initial_state = puzzles.iloc[0, 2]
baseline_solution = sample_submission.loc[0, 'moves']

In [ ]:
state = apply_sequence(baseline_solution, all_moves['cube_2/2/2'], initial_state)
np.array_equal(state, solution_state)

True

# Polytope Permutation - A* Algorithm

In [ ]:
puzzles

,puzzle_type,solution_state,initial_state,num_wildcards
id,,,,
0,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[D, E, D, A, E, B, A, B, C, A, C, A, D, C, D, ...",0
1,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[D, E, C, B, B, E, F, A, F, D, B, F, F, E, B, ...",0
2,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[E, F, C, C, F, A, D, D, B, B, A, F, E, B, C, ...",0
3,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[A, C, E, C, F, D, E, D, A, A, F, A, B, D, B, ...",0
4,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[E, D, E, D, A, E, F, B, A, C, F, D, F, D, C, ...",0
...,...,...,...,...
393,globe_3/33,"[A, A, A, A, A, A, C, C, C, C, C, C, E, E, E, ...","[D, D, L, A, P, E, R, U, U, C, S, R, J, B, E, ...",0
394,globe_3/33,"[A, A, A, A, A, A, C, C, C, C, C, C, E, E, E, ...","[V, L, N, G, B, V, R, E, H, A, K, S, I, N, G, ...",0
395,globe_3/33,"[N0, N1, N2, N3, N4, N5, N6, N7, N8, N9, N10, ...","[N12, N219, N227, N198, N4, N208, N214, N245, ...",0


In [ ]:
import json

# Converting the string representation of allowed_moves to dictionary
puzzle_info_df = pd.read_csv('puzzle_info.csv')
puzzle_info_df['allowed_moves'] = puzzle_info_df['allowed_moves'].apply(lambda x: json.loads(x.replace("'", '"')))

# Selecting an example puzzle type and displaying its allowed moves
example_puzzle_type = puzzle_info_df['puzzle_type'].iloc[0]
example_allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == example_puzzle_type]['allowed_moves'].iloc[0]

example_puzzle_type

'cube_2/2/2'

In [ ]:
puzzle_info_df

,puzzle_type,allowed_moves
0,cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
1,cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ..."
2,cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
3,cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
4,cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
5,cube_7/7/7,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
6,cube_8/8/8,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
7,cube_9/9/9,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
8,cube_10/10/10,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
9,cube_19/19/19,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."


In [ ]:
# cube_2/2/2 의 'allowed_moves'를 확인해 보자
pd.DataFrame(example_allowed_moves).T

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
f0,0,1,19,17,6,4,7,5,2,9,...,14,15,16,20,18,21,10,8,22,23
f1,18,16,2,3,4,5,6,7,8,0,...,12,14,22,17,23,19,20,21,11,9
r0,0,5,2,7,4,21,6,23,10,8,...,1,15,16,17,18,19,20,14,22,12
r1,4,1,6,3,20,5,22,7,8,9,...,14,0,17,19,16,18,15,21,13,23
d0,0,1,2,3,4,5,18,19,8,9,...,10,11,16,17,14,15,22,20,23,21
d1,1,3,0,2,16,17,6,7,4,5,...,14,15,12,13,18,19,20,21,22,23


In [ ]:
# Parsing the initial_state and solution_state columns
# Converting the semicolon-separated string values into lists of colors
puzzles_df = pd.read_csv('puzzles.csv')
puzzles_df['parsed_initial_state'] = puzzles_df['initial_state'].apply(lambda x: x.split(';'))
puzzles_df['parsed_solution_state'] = puzzles_df['solution_state'].apply(lambda x: x.split(';'))

# Displaying the modified dataframe with parsed states
puzzles_df[['id', 'puzzle_type', 'parsed_initial_state', 'parsed_solution_state']].head()

,id,puzzle_type,parsed_initial_state,parsed_solution_state
0,0,cube_2/2/2,"[D, E, D, A, E, B, A, B, C, A, C, A, D, C, D, ...","[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ..."
1,1,cube_2/2/2,"[D, E, C, B, B, E, F, A, F, D, B, F, F, E, B, ...","[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ..."
2,2,cube_2/2/2,"[E, F, C, C, F, A, D, D, B, B, A, F, E, B, C, ...","[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ..."
3,3,cube_2/2/2,"[A, C, E, C, F, D, E, D, A, A, F, A, B, D, B, ...","[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ..."
4,4,cube_2/2/2,"[E, D, E, D, A, E, F, B, A, C, F, D, F, D, C, ...","[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ..."


In [ ]:
def apply_move(state, move, inverse=False):

    if inverse:
        # Creating a dictionary to map the original positions to the new positions
        inverse_move = {v: k for k, v in enumerate(move)}
        return [state[inverse_move[i]] for i in range(len(state))]
    else:
        return [state[i] for i in move]

# Testing the function with an example move from the 'cube_2/2/2' puzzle
test_state = puzzles_df['parsed_initial_state'].iloc[0]
test_move = example_allowed_moves['f1']

# Applying the move and its inverse to the test state
applied_state = apply_move(test_state, test_move)
reverted_state = apply_move(applied_state, test_move, inverse=True)

In [ ]:
pd.DataFrame(applied_state).T

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,E,F,D,A,E,B,A,B,C,D,...,D,D,B,F,C,E,B,F,A,A


In [ ]:
pd.DataFrame(reverted_state).T

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,D,E,D,A,E,B,A,B,C,A,...,D,F,F,F,E,E,B,F,B,C


In [ ]:
import heapq
import time

# Heuristic function estimating the cost from the current state to the goal state.
def heuristic(state, goal_state):
    return sum(s != g for s, g in zip(state, goal_state))

In [ ]:
# A* search algorithm with a timeout feature.
## allowed_moves: A dictionary of moves that can be applied to the state.
## timeout: The maximum time (in seconds) allowed for the search.

def a_star_search_with_timeout(initial_state, goal_state, allowed_moves, timeout=300):

    start_time = time.time()
    open_set = []                                     # Priority queue for states to explore
    heapq.heappush(open_set, (0, initial_state, []))  # Each entry: (priority, state, path taken)

    closed_set = set()                                # Set to keep track of already explored states

    while open_set:
        if time.time() - start_time > timeout:
            return None                               # Timeout check

        _, current_state, path = heapq.heappop(open_set)

        if current_state == goal_state:
            return path                               # Goal state reached

        state_tuple = tuple(current_state)
        if state_tuple in closed_set:
            continue                                  # Skip already explored states

        closed_set.add(state_tuple)

        for move_name, move in allowed_moves.items():
            for inverse in [False, True]:             # Consider both move and its inverse
                new_state = apply_move(current_state, move, inverse)
                new_state_tuple = tuple(new_state)
                if new_state_tuple not in closed_set:
                    priority = len(path) + 1 + heuristic(new_state, goal_state)
                    heapq.heappush(open_set, (priority, new_state, path + [(move_name, inverse)]))

In [ ]:
# Testing the A* search algorithm with an example
test_initial_state = puzzles_df['parsed_initial_state'].iloc[0]
test_goal_state = puzzles_df['parsed_solution_state'].iloc[0]
test_allowed_moves = example_allowed_moves
test_allowed_moves

{'f0': [0,
  1,
  19,
  17,
  6,
  4,
  7,
  5,
  2,
  9,
  3,
  11,
  12,
  13,
  14,
  15,
  16,
  20,
  18,
  21,
  10,
  8,
  22,
  23],
 'f1': [18,
  16,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  0,
  10,
  1,
  13,
  15,
  12,
  14,
  22,
  17,
  23,
  19,
  20,
  21,
  11,
  9],
 'r0': [0,
  5,
  2,
  7,
  4,
  21,
  6,
  23,
  10,
  8,
  11,
  9,
  3,
  13,
  1,
  15,
  16,
  17,
  18,
  19,
  20,
  14,
  22,
  12],
 'r1': [4,
  1,
  6,
  3,
  20,
  5,
  22,
  7,
  8,
  9,
  10,
  11,
  12,
  2,
  14,
  0,
  17,
  19,
  16,
  18,
  15,
  21,
  13,
  23],
 'd0': [0,
  1,
  2,
  3,
  4,
  5,
  18,
  19,
  8,
  9,
  6,
  7,
  12,
  13,
  10,
  11,
  16,
  17,
  14,
  15,
  22,
  20,
  23,
  21],
 'd1': [1,
  3,
  0,
  2,
  16,
  17,
  6,
  7,
  4,
  5,
  10,
  11,
  8,
  9,
  14,
  15,
  12,
  13,
  18,
  19,
  20,
  21,
  22,
  23]}

In [ ]:
# Running the A* search to find a solution
a_star_solution = a_star_search_with_timeout(test_initial_state, test_goal_state, test_allowed_moves)
a_star_solution  # Display the solution moves (if found)

[('r1', False), ('f1', True)]

In [ ]:
# Modifying the A* search algorithm to improve efficiency
## Improved heuristic function considering wildcards.
def improved_heuristic_with_wildcards(state, goal_state, num_wildcards):
    mismatches = sum(s != g for s, g in zip(state, goal_state))

    return max(0, mismatches - num_wildcards)

In [ ]:
def improved_a_star_search_with_wildcards(initial_state, goal_state, allowed_moves, num_wildcards, max_depth=30, timeout=100):

    start_time = time.time()
    open_set = []
    heapq.heappush(open_set, (0, initial_state, [], num_wildcards))  # (priority, state, path, remaining wildcards)
    closed_set = set()

    while open_set:
        if time.time() - start_time > timeout:
            return None                                              # Timeout

        _, current_state, path, remaining_wildcards = heapq.heappop(open_set)

        if len(path) > max_depth:                                    # Depth limit
            continue

        if current_state == goal_state or improved_heuristic_with_wildcards(current_state, goal_state, remaining_wildcards) == 0:
            return path

        closed_set.add((tuple(current_state), remaining_wildcards))

        for move_name, move in allowed_moves.items():
            for inverse in [False, True]:
                new_state = apply_move(current_state, move, inverse)
                if (tuple(new_state), remaining_wildcards) not in closed_set:
                    priority = len(path) + 1 + improved_heuristic_with_wildcards(new_state, goal_state, remaining_wildcards)
                    heapq.heappush(open_set, (priority, new_state, path + [(move_name, inverse)], remaining_wildcards))

    return None                                                     # No solution found

In [ ]:
# Running the improved A* search to find a solution
test_num_wildcards = puzzles_df['num_wildcards'].iloc[0]
improved_a_star_solution = improved_a_star_search_with_wildcards(test_initial_state, test_goal_state, test_allowed_moves, test_num_wildcards)
improved_a_star_solution

[('r1', False), ('f1', True)]

In [ ]:
def format_solution_for_submission(puzzle_id, solution_moves):

    formatted_moves = []
    for move, inverse in solution_moves:
        move_str = '-' + move if inverse else move
        formatted_moves.append(move_str)

    # Joining the moves into a single string separated by periods
    return {'id': puzzle_id, 'moves': '.'.join(formatted_moves)}

In [ ]:
# Example: Formatting the solution for the first puzzle in the dataframe for submission
puzzle_id_example = puzzles_df['id'].iloc[0]
formatted_solution = format_solution_for_submission(puzzle_id_example, a_star_solution)
formatted_solution

{'id': 0, 'moves': 'r1.-f1'}

In [ ]:
from tqdm import tqdm

# Solve a set of puzzles using the A* search algorithm.
def solve_puzzles(puzzles_df, puzzle_info_df, sample_submission_df, num_puzzles=None, limit_index=30):

    solutions = []

    # Limit the number of puzzles if specified
    puzzles_to_solve = puzzles_df if num_puzzles is None else puzzles_df.head(num_puzzles)

    for index, row in tqdm(puzzles_to_solve.iterrows(), total=puzzles_to_solve.shape[0], desc="Solving Puzzles"):
        puzzle_id = row['id']
        initial_state = row['parsed_initial_state']
        goal_state = row['parsed_solution_state']
        puzzle_type = row['puzzle_type']
        num_wildcards = row['num_wildcards']
        allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == puzzle_type]['allowed_moves'].iloc[0]

        solution_moves = None

        # Attempt to find a solution
        if index < limit_index:
            solution_moves = improved_a_star_search_with_wildcards(initial_state, goal_state, allowed_moves, num_wildcards)

        # If no solution found, use the sample submission's result (원래 쓰여져있던 데이터를 사용)
        if solution_moves is None:
            solution_moves = sample_submission_df[sample_submission_df['id'] == puzzle_id]['moves'].iloc[0].split('.')
            solution_moves = [(move.replace('-', ''), move.startswith('-')) for move in solution_moves]

        formatted_solution = format_solution_for_submission(puzzle_id, solution_moves)
        solutions.append(formatted_solution)

    return pd.DataFrame(solutions)

In [ ]:
# Solving the first 3 puzzles in the dataset for testing
sample_submission_df = pd.read_csv('sample_submission.csv')

solved_puzzles_df = solve_puzzles(puzzles_df, puzzle_info_df, sample_submission_df, num_puzzles=3)
solved_puzzles_df

Solving Puzzles: 100%|██████████| 3/3 [00:14<00:00,  4.86s/it]


,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1


### Apply to sample data
This algorithm need to be further improved, so we just test in first 30 easy puzzle

In [ ]:
# Solving the first 30 puzzles in the dataset
solved_puzzles_df = solve_puzzles(puzzles_df, puzzle_info_df, sample_submission_df, num_puzzles=None, limit_index=398)
solved_puzzles_df

Solving Puzzles:  70%|███████   | 279/398 [7:25:55<3:20:59, 101.34s/it]

In [ ]:
# save result for submission
output_csv_path = 'solution.csv'
solved_puzzles_df.to_csv(output_csv_path, index=False)
output_csv_path